# Data Querying and Analysis

## Table of Contents
1. Introduction
2. Examples
3. References and Further Reading

<a id='2'></a>
## 1. Introduction

Generative AI can be used in various ways for Natural Language Querying, specifically for converting English to SQL:

1. **Direct Translation**: Converting natural language questions into SQL queries.
2. **Query Optimization**: Improving existing SQL queries based on natural language descriptions.
3. **Schema Understanding**: Interpreting database schemas from natural language descriptions.
4. **Error Correction**: Identifying and fixing errors in SQL queries based on natural language feedback.
5. 1. Query Translation: Converting natural language questions into formal query languages like SQL.
2. Query Generation: Creating complex queries based on high-level descriptions of data needs.
3. Result Interpretation: Translating query results into human-readable explanations.
4. Schema Understanding: Analyzing database schemas to inform query generation.
5. Error Handling: Identifying and explaining errors in queries or results.

Using Gen AI for this task offers several benefits:
- Increased accessibility for non-technical users
- Faster query development
- Reduced errors in complex queries
- Improved data exploration capabilities

**Key Terminology**:
- **Natural Language Processing (NLP)**: The branch of AI that deals with the interaction between computers and humans using natural language.
- **SQL (Structured Query Language)**: A standard language for managing and manipulating relational databases.
- **Schema**: The structure of a database, defining tables, fields, relationships, and constraints.

In [1]:
!pip install openai pandas scikit-learn matplotlib

In [2]:
import openai
import os
import json
import pandas as pd
from openai import OpenAI

# Set up OpenAI API key
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv('API_KEY')
client = OpenAI(api_key=API_KEY)

def clean(dict_variable):
    return next(iter(dict_variable.values()))

<a id='3'></a>
## 2. Example 1: Simple Query Conversion

In [3]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a SQL expert. Convert English queries to SQL."},
        {"role": "user", "content": "Convert this English query to SQL and output in JSON form: Find top 5 customers by revenue."}
    ],
    response_format={"type": "json_object"}
)

print(response.choices[0].message.content)


{
  "sql": "SELECT customer_id, SUM(revenue) AS total_revenue FROM sales GROUP BY customer_id ORDER BY total_revenue DESC LIMIT 5;"
}


In [4]:
schema = """

sales
order_id
customer_id
sales

customer
customer_code
customer_name

"""


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a SQL expert. Convert English queries to SQL."},
        {"role": "user", "content": "Convert this English query to SQL and output in JSON form: Find top 5 customers (customer names) by sales. The schema of this database is: {}".format(schema)}
    ],
    response_format={"type": "json_object"}
)

print(response.choices[0].message.content)


{"query": "SELECT c.customer_name FROM customer c JOIN sales s ON c.customer_code = s.customer_id GROUP BY c.customer_name ORDER BY SUM(s.sales) DESC LIMIT 5;"}


This example demonstrates how Gen AI can convert a simple English query into SQL. The JSON output makes it easy for data engineers to parse and integrate the result into their workflows.

<a id='5'></a>
## 2. Example 2: Query Optimization and Cleaning

In [5]:
bad_query = """
SELECT * FROM orders WHERE order_date > '2023-01-01' AND customer_id IN (SELECT customer_id FROM customers WHERE country = 'USA')
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a SQL expert. Optimize SQL queries based on natural language descriptions."},
        {"role": "user", "content": "Optimize this SQL query and output in JSON form: {} Optimization goal: Improve performance for a large dataset.".format(bad_query)}
    ],
    response_format={"type": "json_object"}
)

print(json.loads(response.choices[0].message.content))

{'optimized_query': "SELECT o.* FROM orders o JOIN customers c ON o.customer_id = c.customer_id WHERE o.order_date > '2023-01-01' AND c.country = 'USA';", 'explanation': 'This query uses a JOIN instead of a subquery for better performance on large datasets. JOINs are typically more efficient than IN clauses, especially when dealing with larger tables, as they leverage indexed columns effectively.'}


In [6]:
bad_query = """
SELECT *
FROM (
    SELECT c3.id, c3.date, c3.amt, c3.prod_name, c3.region, e.emp_name
    FROM region c3
    JOIN employees e ON c3.id = e.id
    WHERE e.active = 1
) AS subquery
JOIN (
    SELECT AVG(s.amt) AS avg_amt
    FROM sales s
) AS subquery2 ON subquery.amt > subquery2.avg_amt;
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a SQL expert. Given a SQL query, make it more readable. You may split the query into multiple steps with temp tables, and add naming conventions / column names. \
            Avoid complex joins and nested CTEs."},
        {"role": "user", "content": "make it more readable. You may split the query into multiple steps with temp tables, and add naming conventions / column names. \
            Avoid complex joins and nested CTEs. Output in JSON form: {}".format(bad_query)}
    ],
    response_format={"type": "json_object"}
)

good_query = clean(json.loads(response.choices[0].message.content))

In [12]:
good_query

[{'temp_table': 'active_employees',
  'query': 'SELECT e.id, e.emp_name FROM employees e WHERE e.active = 1'},
 {'temp_table': 'region_with_employees',
  'query': 'SELECT c3.id, c3.date, c3.amt, c3.prod_name, c3.region, ae.emp_name FROM region c3 JOIN active_employees ae ON c3.id = ae.id'},
 {'temp_table': 'average_sales',
  'query': 'SELECT AVG(s.amt) AS avg_amt FROM sales s'},
 {'final_query': 'SELECT rwe.id, rwe.date, rwe.amt, rwe.prod_name, rwe.region, rwe.emp_name FROM region_with_employees rwe JOIN average_sales as avg ON rwe.amt > avg.avg_amt'}]

In [7]:
for g in good_query:
    print (g)
    print('----')

{'temp_table_name': 'active_employees', 'sql': 'SELECT e.id, e.emp_name FROM employees e WHERE e.active = 1;'}
----
{'temp_table_name': 'region_with_employees', 'sql': 'SELECT r.id, r.date, r.amt, r.prod_name, r.region, ae.emp_name FROM region r JOIN active_employees ae ON r.id = ae.id;'}
----
{'temp_table_name': 'average_sales', 'sql': 'SELECT AVG(s.amt) AS avg_amt FROM sales s;'}
----
{'final_query': 'SELECT rwe.*, avg.avg_amt FROM region_with_employees rwe JOIN average_sales avg ON rwe.amt > avg.avg_amt;'}
----


## 2. Example 3: Query Result Interpretation

In [8]:
# Simulated query result
query_result = """
| product_category | total_sales |
|-------------------|-------------|
| Electronics      | 1500000     |
| Clothing         | 980000      |
| Home & Garden    | 750000      |
| Books            | 320000      |
| Toys             | 280000      |
"""

prompt = f"Interpret the following query result and provide insights. Here's the result:\n{query_result}"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

The query result provides a breakdown of total sales across five product categories. Here's a detailed interpretation and insights based on these results:

### 1. **Highest Performing Category: Electronics**
   - **Sales:** $1,500,000
   - **Insights:** Electronics significantly outperformed all other categories, generating more than 1.5 times the sales of the next highest category, Clothing. This could indicate a strong consumer demand for electronic products, potential market trends towards gadgets and technology, or effective promotional strategies in this category. 

### 2. **Second Place: Clothing**
   - **Sales:** $980,000
   - **Insights:** Clothing sales are substantial but comparatively lower than electronics. This sector may be influenced by seasonal trends, fashion cycles, and consumer preferences. A consistent marketing strategy, such as seasonal sales or promotions, could further boost this category.

### 3. **Home & Garden**
   - **Sales:** $750,000
   - **Insights:** Thi

<a id='4'></a>
## 2. Example 4: Generating Insights from Query Results

In [9]:
prompt = """Based on the following query result, suggest 3 relevant follow-up questions for further analysis:
{
  "category": ["Electronics", "Clothing", "Books"],
  "total_sales": [500000, 300000, 100000]
}"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

Here are three relevant follow-up questions for further analysis based on the query result:

1. **What are the trends in sales over time for each category?**
   - Understanding how sales have changed over different time periods (monthly, quarterly, yearly) could provide insights into seasonality, consumer behavior, and potential areas for growth or decline.

2. **How do customer demographics and preferences differ across these categories?**
   - Analyzing whether certain demographics (age, gender, location) are more likely to purchase from specific categories can help tailor marketing strategies and product offerings.

3. **What are the top-selling products within each category, and what factors contribute to their success?**
   - Identifying which specific products are driving sales within each category could reveal consumer preferences and key trends, as well as inform inventory and marketing decisions.


This example demonstrates how Gen AI can assist in query optimization, a crucial skill for data engineers working with large datasets and complex database structures.

<a id='7'></a>
## 3. References and Further Reading

1. OpenAI API Documentation: https://platform.openai.com/docs/
2. "Natural Language to SQL: A Survey" by Xu et al. (2022): https://arxiv.org/abs/2201.00307
3. "Improving Text-to-SQL Evaluation Methodology" by Zhong et al. (2020): https://arxiv.org/abs/1806.09029
4. SQL Performance Tuning by Peter Gulutzan and Trudy Pelzer
5. "Fundamentals of Database Systems" by Ramez Elmasri and Shamkant Navathe